This is an attempt to infuse the data with another data from same domain i.e. infusing the common lit data with data from the [CBT dataset](https://huggingface.co/datasets/cbt).

the idea is that since the CBT also contains data from childrens book so the language model trained with this data would be like a child from grade 3-12 . 

In [ ]:
!pip install fastcore

In [ ]:
!yes Y | conda install -c huggingface -c conda-forge datasets

In [ ]:
from typing import List
from itertools import chain
from datasets import load_dataset
from fastcore.all import *
from pandas.core.frame import DataFrame
import numpy as np
import pandas as pd

## Read the data

In [ ]:
dataset = load_dataset("cbt", "CN")

In [ ]:
dataset

Getting the sentences(20 sentences from each book).

In [ ]:
dataset.column_names

In [ ]:
train = L(dataset['train']['sentences'][:2])

In [ ]:
train

## Preprocessing
Initially the sentences are present as list of lists.
joining the sentences in each inner list into a bigger string as this is what will be fed to teh language model

In [ ]:
def list2str(x):    return ''.join(x)

In [ ]:
train.map?

In [ ]:
train.map(list2str)

In [ ]:
train = L(dataset['train']['sentences'])
test = L(dataset['test']['sentences'])
validation = L(dataset['validation']['sentences'])

In [ ]:
train_processed = list(train.map(list2str))
test_processed = list(test.map(list2str))
valid_processed = list(test.map(list2str))

In [ ]:
train_processed[:2]

joining the lists into a single dataframe

In [ ]:
len(train_processed)

In [ ]:
#big_list = list(chain(train_processed, test_processed, valid_processed))

In [ ]:
#len(big_list)

In [ ]:
df_cbt_train = pd.DataFrame(train_processed, columns=['excerpt'])
df_cbt_train['target'] = np.nan
df_cbt_test = pd.DataFrame(test_processed, columns=['excerpt'])
df_cbt_test['target'] = np.nan
df_cbt_valid = pd.DataFrame(valid_processed, columns=['excerpt'])
df_cbt_valid['target'] = np.nan

In [ ]:
#df_cbt.shape

In [ ]:
df_cbt_train.head(2)

## Pre-processing commonlit data prior to concatenation

In [ ]:
df_aug_comlit = pd.read_csv('../input/common-lit-datset-with-synonym-replacement/aug_df.csv')
df_train_vanilla_comlit = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
df_test_vanilla_comlit = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

functions to drop and concatenate

In [ ]:
def drop_cols(df: DataFrame, cols: List) -> DataFrame:
    """drop the given list of columns from the dataframe"""
    return df.drop(cols, axis=1)

def concat_dfs(dfs: List) -> DataFrame:
    """concatenate two dataframes"""
    return pd.concat(dfs)

In [ ]:
df_aug_comlit = drop_cols(df_aug_comlit, ['Unnamed: 0', 
                                                  'id',
                                                  'url_legal',
                                                  'license', 
                                                  'standard_error'])

In [ ]:
df_train_vanilla_comlit = drop_cols(df_train_vanilla_comlit, ['id',
                                                              'url_legal',
                                                              'license', 
                                                              'standard_error'])

In [ ]:
df_aug_comlit.head(1)

In [ ]:
df_train_vanilla_comlit.head(1)

## Infusing the CBT excerpt with the commonlit excerpt
* Preparing two versions-->
  * with augmented commonlit
  * with train set of vanilla commonlit

In [ ]:
df_aug_comlit_infused_with_cbt = concat_dfs([df_aug_comlit, df_cbt_train])
df_vanilla_comlit_train_infused_with_cbt = concat_dfs([df_train_vanilla_comlit, df_cbt_train])

In [ ]:
df_aug_comlit_infused_with_cbt.shape, df_vanilla_comlit_train_infused_with_cbt.shape

In [ ]:
df_aug_comlit_infused_with_cbt.head(3)

In [ ]:
df_aug_comlit_infused_with_cbt.tail()

In [ ]:
df_vanilla_comlit_train_infused_with_cbt.head(3)

## saving both the dataframes

In [ ]:
df_aug_comlit_infused_with_cbt.to_csv('aug_comlit_infused_with_cbt.csv')
df_vanilla_comlit_train_infused_with_cbt.to_csv('vanilla_comlit_train_infused_with_cbt.csv')
df_cbt_train.to_csv('cbt_train.csv')
df_cbt_test.to_csv('cbt_test.csv')
df_cbt_valid.to_csv('cbt_valid.csv')